In [20]:
import os
import json
import pandas as pd
from glob import glob
from eda_support_functions import *

In [21]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_info_columns', 1000)

In [22]:
RUNS_IDS = ['20240720_111518']

In [23]:
parameters_df = load_metadata('parameters', '/Users/iliasx/Documents/GitHub/box-office-prediction/metadata')
experiment_df = load_metadata('metrics', '/Users/iliasx/Documents/GitHub/box-office-prediction/metadata')

In [24]:
experiment_df

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,ROC AUC Score,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2
0,20240719_214914,2024-07-19 22:31:22,decision_tree_classifier,binary_classification,small_productions,random_search,True,none,RobustScaler,0.104955,0.044046,None,0.573385,0.571429,0.653061,0.561404,0.603774,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20240719_214914,2024-07-19 22:13:12,lightgbm_classifier,multi_class_classification,large_productions,random_search,False,none,StandardScaler,0.042468,30.812793,None,NaN,0.496970,0.424794,0.496970,0.440297,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20240720_111518,2024-07-20 11:33:54,dummy_classifier,binary_classification,medium_productions,random_search,False,none,StandardScaler,0.155938,0.051236,None,0.509345,0.526316,0.411594,0.423881,0.417647,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20240720_111518,2024-07-20 11:33:56,random_forest_classifier,binary_classification,medium_productions,random_search,False,none,RobustScaler,0.019995,2.090855,None,0.693565,0.692584,0.600000,0.698507,0.645517,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20240720_111518,2024-07-20 11:22:43,xgboost_classifier,multi_class_classification,medium_productions,random_search,True,none,StandardScaler,0.155938,4.184793,None,NaN,0.491304,0.440050,0.491304,0.460799,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,20240719_214914,2024-07-19 22:04:59,xgboost_classifier,multi_class_classification,medium_productions,random_search,True,none,RobustScaler,0.058246,12.487918,None,NaN,0.501449,0.444184,0.501449,0.467742,NaN,NaN,NaN,NaN,NaN,NaN,NaN
912,20240719_170554,2024-07-19 20:30:07,lightgbm_classifier,multi_class_classification,full,random_search,True,none,RobustScaler,0.036971,358.592392,None,NaN,0.552885,0.488654,0.552885,0.493247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
913,20240719_170554,2024-07-19 17:28:03,lightgbm_classifier,binary_classification,large_productions,random_search,False,none,RobustScaler,0.036971,62.698922,None,0.641836,0.644444,0.631818,0.594017,0.612335,NaN,NaN,NaN,NaN,NaN,NaN,NaN
914,20240719_170554,2024-07-19 17:38:17,logistic_regression,binary_classification,medium_productions,random_search,True,complex,MinMaxScaler,0.141615,18.903358,None,0.669872,0.685507,0.668016,0.550000,0.603291,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
experiment_df.groupby('run_id').aggregate({'run_id': 'count',
                                          'dataset_name' : 'nunique',
                                          'model_type' : 'nunique',
                                          'timestamp' : 'max'}
                                          ).rename(columns={'run_id': 'count'}).sort_values('timestamp', ascending=False)

,count,dataset_name,model_type,timestamp
run_id,,,,
20240720_111518,320,4,13,2024-07-20 11:42:02
20240720_111340,7,1,7,2024-07-20 11:13:59
20240719_232009,13,1,13,2024-07-19 23:55:01
20240719_231620,12,2,6,2024-07-19 23:17:37
20240719_231427,12,2,6,2024-07-19 23:15:42
20240719_225631,2,2,1,2024-07-19 22:58:14
20240719_214914,272,4,11,2024-07-19 22:31:25
20240719_214632,6,1,6,2024-07-19 21:47:09
20240719_170554,272,4,11,2024-07-19 21:41:48


In [26]:
parameters_df = parameters_df[parameters_df['run_id'].isin(RUNS_IDS)]
experiment_df = experiment_df[experiment_df['run_id'].isin(RUNS_IDS)]

In [27]:
# parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'model_type', 'model__units', 'model__num_layers', 'model__dropout', 'epochs','variance_threshold','class_weight']].sort_values(by=['problem_type','dataset_name','has_outliers_removed'])

In [28]:
parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'model_type', 'scaler','variance_threshold','class_weight','C','min_child_samples','solver','num_leaves','n_estimators']].sort_values(by=['model_type','problem_type','dataset_name','has_outliers_removed'])

,problem_type,dataset_name,has_outliers_removed,feature_engineering,model_type,scaler,variance_threshold,class_weight,C,min_child_samples,solver,num_leaves,n_estimators
109,binary_classification,full,False,none,decision_tree_classifier,StandardScaler,0.011617,balanced,NaN,NaN,NaN,NaN,NaN
253,binary_classification,full,False,complex,decision_tree_classifier,StandardScaler,0.011617,balanced,NaN,NaN,NaN,NaN,NaN
48,binary_classification,full,True,none,decision_tree_classifier,StandardScaler,0.011617,balanced,NaN,NaN,NaN,NaN,NaN
896,binary_classification,full,True,complex,decision_tree_classifier,StandardScaler,0.155938,balanced,NaN,NaN,NaN,NaN,NaN
281,binary_classification,large_productions,False,complex,decision_tree_classifier,StandardScaler,0.011617,balanced,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,regression,medium_productions,True,none,xgboost_regressor,RobustScaler,0.019995,NaN,NaN,NaN,NaN,NaN,566.0
714,regression,small_productions,False,none,xgboost_regressor,RobustScaler,0.019995,NaN,NaN,NaN,NaN,NaN,566.0
859,regression,small_productions,False,complex,xgboost_regressor,RobustScaler,0.019995,NaN,NaN,NaN,NaN,NaN,566.0
805,regression,small_productions,True,complex,xgboost_regressor,StandardScaler,0.146399,NaN,NaN,NaN,NaN,NaN,1394.0


In [29]:
parameters_df

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,ccp_alpha,class_weight,criterion,max_depth,max_features,max_leaf_nodes,min_impurity_decrease,min_samples_leaf,min_samples_split,min_weight_fraction_leaf,monotonic_cst,random_state,splitter,boosting_type,colsample_bytree,importance_type,learning_rate,min_child_samples,min_child_weight,min_split_gain,n_estimators,n_jobs,num_leaves,objective,reg_alpha,reg_lambda,subsample,subsample_for_bin,subsample_freq,constant,strategy,bootstrap,max_samples,oob_score,verbose,warm_start,base_score,booster,callbacks,colsample_bylevel,colsample_bynode,device,early_stopping_rounds,enable_categorical,eval_metric,feature_types,gamma,grow_policy,interaction_constraints,max_bin,max_cat_threshold,max_cat_to_onehot,max_delta_step,max_leaves,missing,monotone_constraints,multi_strategy,num_parallel_tree,sampling_method,scale_pos_weight,tree_method,validate_parameters,verbosity,num_class,copy_X,fit_intercept,positive,C,dual,intercept_scaling,l1_ratio,max_iter,multi_class,penalty,solver,tol,break_ties,cache_size,coef0,decision_function_shape,degree,kernel,probability,shrinking,quantile
2,20240720_111518,2024-07-20 11:33:54,dummy_classifier,binary_classification,medium_productions,random_search,False,none,StandardScaler,0.155938,0.051236,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stratified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20240720_111518,2024-07-20 11:33:56,random_forest_classifier,binary_classification,medium_productions,random_search,False,none,RobustScaler,0.019995,2.090855,None,0.0,balanced,gini,8.0,sqrt,NaN,0.0,1.0,27.0,0.0,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20240720_111518,2024-07-20 11:22:43,xgboost_classifier,multi_class_classification,medium_productions,random_search,True,none,StandardScaler,0.155938,4.184793,None,NaN,balanced,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,NaN,NaN,None,0.160309,NaN,NaN,NaN,1230.0,-1.0,NaN,multi:softprob,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20240720_111518,2024-07-20 11:26:09,lightgbm_classifier,binary_classification,large_productions,random_search,True,none,StandardScaler,0.091850,3.668635,None,NaN,balanced,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,gbdt,1.0,split,0.090167,20.0,0.001,0.0,100.0,-1.0,90.0,None,0.0,0.0,1.0,200000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,20240720_111518,2024-07-20 11:20:42,logistic_regression,binary_classification,small_productions,random_search,False,complex,RobustScaler,0.019995,1.592516,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,0.381154,False,1.0,NaN,1000.0,multinomial,l1,saga,0.0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [30]:
parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'n_estimators', 'max_depth', 'learning_rate', 'variance_threshold','class_weight']].sort_values(by=['problem_type','dataset_name','has_outliers_removed'])

,problem_type,dataset_name,has_outliers_removed,feature_engineering,n_estimators,max_depth,learning_rate,variance_threshold,class_weight
109,binary_classification,full,False,none,NaN,27.0,NaN,0.011617,balanced
245,binary_classification,full,False,none,NaN,NaN,NaN,0.019995,None
253,binary_classification,full,False,complex,NaN,27.0,NaN,0.011617,balanced
475,binary_classification,full,False,none,430.0,24.0,0.090167,0.091850,balanced
480,binary_classification,full,False,none,1394.0,21.0,NaN,0.146399,balanced
...,...,...,...,...,...,...,...,...,...
704,regression,small_productions,True,none,NaN,8.0,NaN,0.011617,NaN
805,regression,small_productions,True,complex,1394.0,30.0,0.075908,0.146399,NaN
867,regression,small_productions,True,complex,100.0,30.0,0.075908,0.146399,None
893,regression,small_productions,True,complex,NaN,NaN,NaN,0.159309,NaN


In [31]:
# Assuming the DataFrame is named experiment_df
grouped = experiment_df[experiment_df['problem_type'] == 'regression'].groupby(
    ['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering'])

# Get the index of the minimum MAPE in each group
idx = grouped['MAPE'].idxmin()

# Select the rows that correspond to the minimum MAPE in each group
best_models = experiment_df.loc[idx]

# Display the desired columns
result_reg = best_models[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering','model_type', 'scaler', 'MAPE', 'MAE', 'R2']]

In [32]:
#pretty print
print(result_reg.to_string(index=False))

problem_type       dataset_name  has_outliers_removed feature_engineering              model_type         scaler       MAPE          MAE        R2
  regression               full                 False             complex decision_tree_regressor StandardScaler 186.667563 7.486877e+07  0.419972
  regression               full                 False                none       xgboost_regressor   RobustScaler  89.718674 8.252557e+07  0.326098
  regression               full                  True             complex decision_tree_regressor StandardScaler   5.202556 7.979975e+07  0.384334
  regression               full                  True                none       xgboost_regressor   RobustScaler   4.805599 7.993184e+07  0.396658
  regression  large_productions                 False             complex       xgboost_regressor   RobustScaler   4.031964 1.461477e+08  0.393141
  regression  large_productions                 False                none      lightgbm_regressor   RobustScaler   3.4

In [33]:
# Assuming the DataFrame is named experiment_df
grouped = experiment_df[experiment_df['problem_type'] != 'regression'].groupby(
    ['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering'])

# Get the index of the minimum MAPE in each group
idx = grouped['F1 Score'].idxmin()

# Select the rows that correspond to the minimum MAPE in each group
best_models = experiment_df.loc[idx]

# Display the desired columns
result_class = best_models[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering','model_type', 'scaler', 'F1 Score']]

In [34]:
print(result_class.to_string(index=False))

              problem_type       dataset_name  has_outliers_removed feature_engineering               model_type         scaler  F1 Score
     binary_classification               full                 False             complex         dummy_classifier   RobustScaler  0.425656
     binary_classification               full                 False                none         dummy_classifier StandardScaler  0.444115
     binary_classification               full                  True             complex         dummy_classifier   RobustScaler  0.463348
     binary_classification               full                  True                none         dummy_classifier StandardScaler  0.455464
     binary_classification  large_productions                 False             complex           svm_classifier   RobustScaler  0.323944
     binary_classification  large_productions                 False                none         dummy_classifier StandardScaler  0.467249
     binary_classification  large_

In [35]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='binary_classification', metric='F1 Score', metric_agg='max')


In [36]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='multi_class_classification', metric='F1 Score', metric_agg='max')

In [37]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='regression', metric='MAPE', metric_agg='min')

In [38]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='regression', metric='R2', metric_agg='max')